<h1>Importar librerías ecesarias</h1>

In [ ]:
import json
import pandas as pd

import numpy as np
import tensorflow as tf

In [ ]:
#from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model

<h1>Cargar datos</h1>

<h3 style="color:crimson">Cargar modelos</h3>

In [ ]:
model = load_model("../Modelos/s2q/spanish_to_quechua")

<h3 style="color:crimson">Cargar valores adicionales</h3>

In [ ]:
batch_size = 64 # tamño de los lotes para entrenamiento
epochs = 100 # Número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = 10000
#data_path = "../Datos/spa-eng/spa.txt" # 124548 lines

In [ ]:
with open("../Saved_Models/input_token_index.txt", "r") as f:
    input_token_index = json.load(f)

with open("../Saved_Models/target_token_index.txt", "r") as f:
    target_token_index = json.load(f)

with open("../Saved_Models/encoder_input_data.npy", "rb") as f:
    encoder_input_data = np.load(f)

In [ ]:
df_input_texts = pd.read_csv("../Saved_Models/input_texts.csv")

input_texts = []
for index, word in df_input_texts.iterrows():    
    #input_texts.append(word[1])
    input_texts.append(word)

<h1>Modelo de inferencia</h1>

In [ ]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup taken index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

<h1>Probar el modelo</h1>

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Oración de entrada:", input_texts[seq_index])
    print("Oración decodificada:", decoded_sentence)